# ダジャレ検索 - from dajare.jp

In [1]:
import requests
from bs4 import BeautifulSoup
import urllib
import json
# import requests_cache
import re
# requests_cache.install_cache('dajare')

In [2]:
s = 'みかん'
s_quote = urllib.parse.quote(s)
r = requests.get(f'https://dajare.jp/keyword/{s_quote}/')

In [3]:
bs = BeautifulSoup(r.text)

In [4]:
l = []
for x in bs.find_all('tr'):
    dajare = x.find("td", class_='ListWorkBody')
    if not dajare:
        continue
    score = x.find("td", class_="ListWorkScore")
    l.append({'text':dajare.text, 'score':float(score.text.split(' (')[0])})

In [5]:
sorted(l, key=lambda x: x['score'], reverse=True)

[{'text': 'みかん星は、まだ未完成‼️', 'score': 4.0},
 {'text': '南関東では皆、みかん党', 'score': 4.0},
 {'text': 'みかんが三日も見っかんねぇ', 'score': 4.0},
 {'text': 'ミカン製品が未完成品だった。', 'score': 4.0},
 {'text': '関空で、ミカン食う。', 'score': 4.0},
 {'text': '阿部なつみ監修、夏みかん集！', 'score': 3.8},
 {'text': '奈津美噛んだ夏ミカンだ', 'score': 3.5},
 {'text': '私のみかんが３日間、見っかんない・・・', 'score': 3.5},
 {'text': 'アルミ缶の上にあるミカン', 'score': 3.2},
 {'text': 'みかん農家のみ官能か？', 'score': 3.0},
 {'text': '未完、無効化したのでミカン剥こうか？', 'score': 3.0},
 {'text': '奈津美、環奈の好物は夏みかんなの', 'score': 3.0},
 {'text': '官能みかん農園', 'score': 3.0},
 {'text': '和歌山産みかん汁には、香山、酸味感じる', 'score': 3.0},
 {'text': '近江観光で王、ミカン乞う', 'score': 3.0},
 {'text': 'みかんが三日も見っかんない', 'score': 3.0},
 {'text': 'オランジーナでホラン自慰な', 'score': 3.0},
 {'text': 'ミカンのみ完', 'score': 3.0},
 {'text': '「お菓子、ミカンだ...」と、可笑しみ噛んだ', 'score': 3.0},
 {'text': 'ミカン農家のみ官能か？', 'score': 3.0},
 {'text': '関与した、みかんよ。', 'score': 3.0},
 {'text': 'いい伊予柑、いいよ噛んで、', 'score': 3.0},
 {'text': 'ミカンの実、噛んだ。……', 'score': 3.0},
 {'text': '未感動なみかんどう？', 'score': 

# ダジャレクローラー - dajare.jp

In [6]:
bs=BeautifulSoup(requests.get('https://dajare.jp/').text)

In [7]:
rex = re.compile(r'/category/*')

link_list = set()
for x in bs.find_all('a', href=rex):
    if len(x.get('href').split('/'))==6:
        link_list.add(x.get('href'))

In [8]:
rex = re.compile(r'/works/*')

dajare_link_list = set()
for x in link_list:
    x = x.split('/')
    data={'ViewQuantity': 1000, 'CategoryLarge': x[2], 'CategoryMiddle': x[3], 'CategorySmall': x[4], 'ViewOffset':0}
    bs = BeautifulSoup(requests.post('https://dajare.jp/search/', data=data).text)
    for u in bs.find_all('a', href=rex):
        dajare_link_list.add(u.get('href'))
    break

In [9]:
mean_rex = re.compile(r'平均スコア：([0-9\.]+)')
dev_rex = re.compile(r'だじゃれ偏差値：([0-9\.]+)')

for u in list(dajare_link_list)[100:]:
    url = f'https://dajare.jp{u}'
    bs = BeautifulSoup(requests.get(url).text)
    bs = bs.find(id='PanelContentMain')
    
    # dajare text, author
    panel = bs.find(class_='PanelBox')
    text = panel.find('span').text
    author = panel.find('a').text
    author_link = panel.find('a').get('href')
    
    # dajare meta data
    fieldset = bs.find_all('fieldset')
    eval_list = []
    for field in fieldset:
        legend = field.find('legend')
        if not legend:
            if '平均スコア' in field.text:
                # dajare score
                x = field.text.replace('\n', '')
                mean_score = float(mean_rex.findall(x)[0])
                dev_score = float(dev_rex.findall(x)[0])
            else:
                # dajare evaluation scores
                eval_row = {}
                for row in field.find_all(class_='PanelWorkEvaluationList'):
                    col = row.find('dt').text.strip()
                    if col == '投稿者':
                        eval_row.update({'author': {'link': row.find('a').get('href'), 'text':row.text.strip()}})
                    if col == 'スコア':
                        eval_row.update({'score': float(row.text.replace('スコア\n',''))})
                    if col =='評価投稿時刻':
                        eval_row.update({'datetime': row.text.replace('評価投稿時刻','').strip()})
                eval_list.append(eval_row)
        elif 'カテゴリー' in legend.text:
            # dajare category
            category=[{'link':x.get("href"), 'text':x.text} for x in set(field.find_all('a'))]
        elif 'タグ' in legend.text:
            # dajare tags
            tag = [{'link': x.get("href"), 'text':x.text} for x in field.find_all('a')]

    d = {'url': url,'text':text, 'author':author, 'author_link':author_link,'mean_score': mean_score, 'deviation_score': dev_score, 'category':category, 'tag':tag, 'eval_list':eval_list}
    print(json.dumps(d, indent=4, ensure_ascii=False))
    break

# ダジャレクローラー - dajarenavi.net

In [10]:
# menu/0/m_01____a.htm ~ menu/0/m_91____wa.htm
bs = BeautifulSoup(requests.get('https://dajarenavi.net/index.htm').text)

# a,i,u,e,o,ka,ki,...
aiueo_link_list = []
for x in bs.find_all('tr', bgcolor="#000000"):
    for y in x.find_all('a'):
        aiueo_link_list.append(y.get('href'))
    break
        
# a-ka, sa-wa, ...
fukugo_link_list = []
for keyword_link in aiueo_link_list:
    bs = BeautifulSoup(requests.get(f'https://dajarenavi.net/{keyword_link}').text)
    for x in bs.find_all('tr', bgcolor="#000000"):
        for y in x.find_all('a'):
            fukugo_link_list.append(y.get('href'))
    break

# art, arm, ...
keyword_link_list=[]
for link in fukugo_link_list:
    bs = BeautifulSoup(requests.get(f'https://dajarenavi.net/menu/0/{link}').text)
    for x in bs.find_all('tr', bgcolor="#000000"):
        for a in x.find_all('a'):
            if not a.get('target'):
                keyword_link_list.append(a.get('href'))
    break

In [11]:
dajare = []
for link in keyword_link_list:
    url = f'https://dajarenavi.net/{link.replace("../../","")}'
    r = requests.get(url)
    r.encoding = 'shift-jis'
    bs = BeautifulSoup(r.text)
    keyword = bs.find(class_="kw_title").find('span').text.strip()
    for x in bs.find_all(class_="dajare"):
        d = {'url': url,'text':x.text.strip(), 'author':'dajarenavi', 'author_link':'dajarenavi.net','mean_score': 0., 'deviation_score': 0., 'category':[], 'tag':[{'link':link.replace("../..",""),'text':keyword}], 'eval_list':[]}
        dajare.append(d)
    break

In [12]:
print(json.dumps(dajare[:3],indent=4, ensure_ascii=False))

[
    {
        "url": "https://dajarenavi.net/kw/a/art.htm",
        "text": "アートが分からない奴には、ラリアート！",
        "author": "dajarenavi",
        "author_link": "dajarenavi.net",
        "mean_score": 0.0,
        "deviation_score": 0.0,
        "category": [],
        "tag": [
            {
                "link": "/kw/a/art.htm",
                "text": "アート"
            }
        ],
        "eval_list": []
    },
    {
        "url": "https://dajarenavi.net/kw/a/art.htm",
        "text": "アートって、あーとっても素敵！",
        "author": "dajarenavi",
        "author_link": "dajarenavi.net",
        "mean_score": 0.0,
        "deviation_score": 0.0,
        "category": [],
        "tag": [
            {
                "link": "/kw/a/art.htm",
                "text": "アート"
            }
        ],
        "eval_list": []
    },
    {
        "url": "https://dajarenavi.net/kw/a/art.htm",
        "text": "アートって、あーとっても美しい！",
        "author": "dajarenavi",
        "author_link": "dajarenavi.net",
       

# ダジャレクローラー - dajare.net

In [13]:
rex = re.compile(r'^[0-9]+.htm')

r = requests.get('http://www.dajare.net/')
r.encoding='shift-jis'
bs =  BeautifulSoup(r.text)
dump_list = ['{:03}.htm'.format(x) for x in range(1,50+1)]+[x.get('href') for x in bs.find_all('a', href=rex)]

In [14]:
for x in dump_list:
    url = f'http://www.dajare.net/{x}'
    r = requests.get(url)
    r.encoding='shift-jis'
    bs =  BeautifulSoup(r.text)
    bs = bs.find('table')
    l = []
    for row in bs.find('td').text.strip().split('\n'):
        row = row.strip().split(' ')
        if row[0].startswith('['):
            author = row[1]
        else:
            text = ' '.join(row).replace('\u3000','')
            l.append({'text':text, 'url': url, 'author':author,  'author_link':'dajare.net', 'mean_score': 0., 'deviation_score': 0., 'category':[], 'tag':[], 'eval_list':[]})
    break

In [15]:
l[:3]

[{'text': '仙台！！仙台！！仙台！！「うるせぇんだい！！」',
  'url': 'http://www.dajare.net/001.htm',
  'author': '水族館',
  'author_link': 'dajare.net',
  'mean_score': 0.0,
  'deviation_score': 0.0,
  'category': [],
  'tag': [],
  'eval_list': []},
 {'text': 'ふとんがふっとんだ〜〜〜！',
  'url': 'http://www.dajare.net/001.htm',
  'author': '謎の少年',
  'author_link': 'dajare.net',
  'mean_score': 0.0,
  'deviation_score': 0.0,
  'category': [],
  'tag': [],
  'eval_list': []},
 {'text': 'このキャベツ虫がついてる。',
  'url': 'http://www.dajare.net/001.htm',
  'author': 'ヤマジー',
  'author_link': 'dajare.net',
  'mean_score': 0.0,
  'deviation_score': 0.0,
  'category': [],
  'tag': [],
  'eval_list': []}]

# ダジャレクローラー - www.biwa.ne.jp/~aki-ina/gyagu.html

In [16]:
url = 'http://www.biwa.ne.jp/~aki-ina/gyagu.html'
r = requests.get(url)
r.encoding='utf-8'
bs =  BeautifulSoup(r.text)

l=[]
for ul in bs.find_all('ul'):
    for li in ul.find_all('li'):
        if not li.find('a'):
            l.append({'text':li.text.strip(), 'url': url, 'author':'BAKADAS',  'author_link':'www.biwa.ne.jp/~aki-ina', 'mean_score': 0., 'deviation_score': 0., 'category':[], 'tag':[], 'eval_list':[]})            

In [17]:
l[:3]

[{'text': '生姜が無いって。しょうがないなー',
  'url': 'http://www.biwa.ne.jp/~aki-ina/gyagu.html',
  'author': 'BAKADAS',
  'author_link': 'www.biwa.ne.jp/~aki-ina',
  'mean_score': 0.0,
  'deviation_score': 0.0,
  'category': [],
  'tag': [],
  'eval_list': []},
 {'text': 'あそこのソーダやさんおいしそーだ',
  'url': 'http://www.biwa.ne.jp/~aki-ina/gyagu.html',
  'author': 'BAKADAS',
  'author_link': 'www.biwa.ne.jp/~aki-ina',
  'mean_score': 0.0,
  'deviation_score': 0.0,
  'category': [],
  'tag': [],
  'eval_list': []},
 {'text': 'ロシアの殺しあ。おそろしあ',
  'url': 'http://www.biwa.ne.jp/~aki-ina/gyagu.html',
  'author': 'BAKADAS',
  'author_link': 'www.biwa.ne.jp/~aki-ina',
  'mean_score': 0.0,
  'deviation_score': 0.0,
  'category': [],
  'tag': [],
  'eval_list': []}]

# ダジャレクローラー - dajareshuu.web.fc2.com/

In [18]:
url = 'https://dajareshuu.web.fc2.com/'
r = requests.get(url)
r.encoding='utf-8'
bs =  BeautifulSoup(r.text)

l = []
for tr in bs.find_all('tr'):
    flag = True
    for td in tr.find_all('td'):
        if not td.find('a'):
            if flag:
                keyword = td.text.strip()
                flag = False
            else:
                l.append({'text':td.text.strip(), 'url': url, 'author':'dajareshuu',  'author_link':'dajareshuu.web.fc2.com', 'mean_score': 0., 'deviation_score': 0., 'category':[], 'tag':[{'link':'dajareshuu.web.fc2.com','text':keyword}], 'eval_list':[]})   
                flag = True

In [19]:
l[:3]

[{'text': 'アーカンソーはあかんそうだ',
  'url': 'https://dajareshuu.web.fc2.com/',
  'author': 'dajareshuu',
  'author_link': 'dajareshuu.web.fc2.com',
  'mean_score': 0.0,
  'deviation_score': 0.0,
  'category': [],
  'tag': [{'link': 'dajareshuu.web.fc2.com', 'text': 'アーカンソー'}],
  'eval_list': []},
 {'text': 'あーと少しでアートな作品完成',
  'url': 'https://dajareshuu.web.fc2.com/',
  'author': 'dajareshuu',
  'author_link': 'dajareshuu.web.fc2.com',
  'mean_score': 0.0,
  'deviation_score': 0.0,
  'category': [],
  'tag': [{'link': 'dajareshuu.web.fc2.com', 'text': 'アート'}],
  'eval_list': []},
 {'text': 'アールグレイ茶があーるぐれい',
  'url': 'https://dajareshuu.web.fc2.com/',
  'author': 'dajareshuu',
  'author_link': 'dajareshuu.web.fc2.com',
  'mean_score': 0.0,
  'deviation_score': 0.0,
  'category': [],
  'tag': [{'link': 'dajareshuu.web.fc2.com', 'text': 'アールグレイ'}],
  'eval_list': []}]

# ダジャレクローラー - syuukaizyo.fc2web.com/02.html

In [20]:
url = 'http://syuukaizyo.fc2web.com/02.html'
r = requests.get(url)
r.encoding='shift-jis'
bs =  BeautifulSoup(r.text)

l = []
for x in bs.find('pre').text.split('\n'):
    if x.startswith('・'):
        l.append({'text':x.replace('・','').strip(), 'url': url, 'author':'syuukaizyo',  'author_link':'syuukaizyo.fc2web.com', 'mean_score': 0., 'deviation_score': 0., 'category':[], 'tag':[], 'eval_list':[]})   

In [21]:
l[:3]

[{'text': '布団がふっとんだ',
  'url': 'http://syuukaizyo.fc2web.com/02.html',
  'author': 'syuukaizyo',
  'author_link': 'syuukaizyo.fc2web.com',
  'mean_score': 0.0,
  'deviation_score': 0.0,
  'category': [],
  'tag': [],
  'eval_list': []},
 {'text': 'ハエは速え',
  'url': 'http://syuukaizyo.fc2web.com/02.html',
  'author': 'syuukaizyo',
  'author_link': 'syuukaizyo.fc2web.com',
  'mean_score': 0.0,
  'deviation_score': 0.0,
  'category': [],
  'tag': [],
  'eval_list': []},
 {'text': '内臓がないぞう',
  'url': 'http://syuukaizyo.fc2web.com/02.html',
  'author': 'syuukaizyo',
  'author_link': 'syuukaizyo.fc2web.com',
  'mean_score': 0.0,
  'deviation_score': 0.0,
  'category': [],
  'tag': [],
  'eval_list': []}]

# ダジャレクローラー - kaishaseikatsu.jp

In [22]:
for i in range(0, 2200, 100):
    url='http://archives.kaishaseikatsu.jp/cgi-bin/kaisha2/board_r.cgi?type=kaisha_dajare&next=i&range=100'
    r = requests.get(url)
    r.encoding = 'shift-jis'
    bs = BeautifulSoup(r.text)
    l = []
    for x in bs.find_all('tr', bgcolor="#FBFFB2"):
        l.append({'text':x.find('td').text, 'url': url, 'author':'kaishaseikatsu',  'author_link':'http://archives.kaishaseikatsu.jp', 'mean_score': 0., 'deviation_score': 0., 'category':[], 'tag':[], 'eval_list':[]})   
    print(json.dumps(l[:3],indent=4, ensure_ascii=False))
    break

[
    {
        "text": "寒ぶりア紀",
        "url": "http://archives.kaishaseikatsu.jp/cgi-bin/kaisha2/board_r.cgi?type=kaisha_dajare&next=i&range=100",
        "author": "kaishaseikatsu",
        "author_link": "http://archives.kaishaseikatsu.jp",
        "mean_score": 0.0,
        "deviation_score": 0.0,
        "category": [],
        "tag": [],
        "eval_list": []
    },
    {
        "text": "無論、オムロン",
        "url": "http://archives.kaishaseikatsu.jp/cgi-bin/kaisha2/board_r.cgi?type=kaisha_dajare&next=i&range=100",
        "author": "kaishaseikatsu",
        "author_link": "http://archives.kaishaseikatsu.jp",
        "mean_score": 0.0,
        "deviation_score": 0.0,
        "category": [],
        "tag": [],
        "eval_list": []
    },
    {
        "text": "そばで煮て食えるだけでいい♪",
        "url": "http://archives.kaishaseikatsu.jp/cgi-bin/kaisha2/board_r.cgi?type=kaisha_dajare&next=i&range=100",
        "author": "kaishaseikatsu",
        "author_link": "http://archives.kaishaseikat